In [42]:
import learning2read
from learning2read.b06 import *
from learning2read.proc import Procedure
from learning2read.preprocessing import TotalDataFrame
from importlib import reload
reload(learning2read.preprocessing)
learning2read.reload(learning2read.preprocessing)

<module 'learning2read.preprocessing' from '/tmp2/b06902021/miniconda3/lib/python3.6/site-packages/learning2read/preprocessing.py'>

In [2]:
Data("users").shape, Data("isbn").shape

((278858, 3), (226, 4))

In [3]:
procedure6001 = [
    {
        'class' : 'learning2read.preprocessing.TotalDataFrame',
        'output' : 'df_total',
        'input_data' : ['raw_train', 'raw_implicit', 'raw_test'],
    },
    {
        'class' : 'learning2read.preprocessing.UserRatingSqueeze',
        'output' : 'user_rating',
        'input_data' : 'raw_train',
        'filter_num' : 3,
        'statistics' : ['num','mean','std','skew'],
        'na_policy' : 'median',
    },
    {
        'class' : 'learning2read.preprocessing.BookRatingSqueeze',
        'output' : 'book_rating',
        'input_data' : 'raw_train',
        'filter_num' : 3,
        'statistics' : ['num','mean','std','skew'],
        'na_policy' : 'median',
    },
    {
        'class'  : 'learning2read.preprocessing.UserBookTable',
        'output' : 'df_total_features', # (X,y={-1,0,1,2,...,10})
        'input_data' : ['df_total', 'user_rating', 'book_rating'],
        'na_policy' : None, # should fill it before training
    },
    {
        'class'  : 'learning2read.preprocessing.RowFilter',
        'output' : 'df_train',
        'input_data' : 'df_total_features',
        'func' : r"lambda df : df['Book-Rating']>0",
    }
]

In [4]:
def run_proc(proc, filename, data, cache = True):
    if cache and File.is_readable(filename):
        P = load_pickle(File(filename))
    else:
        P = Procedure(proc, verbose = True)
        P.load_data(data)
        P.run()
        save_pickle(File(filename), P)
    return P
def print_proc(proc): return pd.DataFrame(proc).loc[:,['class','input_data','output']]

In [5]:
pd.DataFrame(procedure6001).loc[:,['class','input_data','output']]

,class,input_data,output
0,learning2read.preprocessing.TotalDataFrame,"[raw_train, raw_implicit, raw_test]",df_total
1,learning2read.preprocessing.UserRatingSqueeze,raw_train,user_rating
2,learning2read.preprocessing.BookRatingSqueeze,raw_train,book_rating
3,learning2read.preprocessing.UserBookTable,"[df_total, user_rating, book_rating]",df_total_features
4,learning2read.preprocessing.RowFilter,df_total_features,df_train


In [50]:
procedure6001 = [
    {
        'class' : 'learning2read.preprocessing.TotalDataFrame',
        'output' : 'df_total',
        'input_data' : ['raw_train', 'raw_implicit', 'raw_test'],
    },
    {
        'class' : 'learning2read.preprocessing.CleanRawBook',
        'output' : 'book_info',
        'input_data' : ['df_total', 'raw_book', 'isbn_table'],
        'na_policy' : 'median',
    },
    {
        'class' : 'learning2read.preprocessing.CleanRawUser',
        'output' : 'user_info',
        'input_data' : ['df_total', 'raw_user'],
        'na_policy' : 'median',
    }
]

P = run_proc(procedure6001, "p6001_1_test", {
        'raw_train': Data('dattrain'),
        'raw_test': Data('dattest'),
        'raw_implicit': Data('datimplicit'),
        'isbn_table': Data('isbn'),
        'raw_book': Data('datbooks'),
        'raw_user': Data('datulatlon'),
    }, True)

P.input_from_var('book_info').sample(20)

,ISBN,ISBN_country_code,ISBN_country,ISBN_lat,ISBN_lon,Year_isna,Year-Of-Publication
141140,0575052805,0,English,38.3345,-86.4641,0,1992.0
337935,9728413246,972,Portugal,40.0333,-7.88963,1,1996.0
39935,0312120494,0,English,38.3345,-86.4641,0,1994.0
26321,0140450378,0,English,38.3345,-86.4641,0,1950.0
9941,0060950668,0,English,38.3345,-86.4641,0,1994.0
23834,0140275428,0,English,38.3345,-86.4641,0,2003.0
229754,0881063940,0,English,38.3345,-86.4641,0,2001.0
324618,8495572060,84,Spain,40.0028,-4.0031,1,1996.0
218668,0842350357,0,English,38.3345,-86.4641,0,1993.0
192358,0770060242,0,English,38.3345,-86.4641,0,1959.0


In [49]:
P.input_from_var('user_info').sample(20)

,User-ID,Lat,Lon,Match,Age,Age_isna
32525,4f3a92d0fc,39.101454,-84.512460,0,33.0,1
71628,ae6a87ccf4,42.590365,-72.309807,0,33.0,1
96800,eb55bad526,37.329709,-121.903160,0,33.0,1
24994,3cfe81c3be,37.779179,-97.467267,0,44.0,0
31367,4c7b4c8b26,59.913269,10.739111,0,40.0,0
1810,04894b501e,45.497216,-73.610364,0,33.0,1
31933,4dc4e31139,41.382894,2.177432,0,21.0,0
43631,6a7eaef68a,45.024950,-79.362552,0,48.0,0
39919,613860a087,29.758938,-95.367697,0,39.0,0
10577,1a2e393212,43.300697,-73.586082,0,62.0,0


In [52]:
P.input_from_var('book_info').loc[lambda df: df.ISBN.str.contains('^978'), :]

,ISBN,ISBN_country_code,ISBN_country,ISBN_lat,ISBN_lon,Year_isna,Year-Of-Publication
338305,9780002000505,0,English,38.3345,-86.4641,1,1996.0
338306,9780006470977,0,English,38.3345,-86.4641,1,1996.0
338307,9780006531777,0,English,38.3345,-86.4641,1,1996.0
338308,9780006710813,0,English,38.3345,-86.4641,1,1996.0
338309,9780020780601,0,English,38.3345,-86.4641,1,1996.0
338310,9780060008253,0,English,38.3345,-86.4641,1,1996.0
338311,9780060173081,0,English,38.3345,-86.4641,1,1996.0
338312,9780060628475,0,English,38.3345,-86.4641,1,1996.0
338313,9780064407663,0,English,38.3345,-86.4641,1,1996.0
338314,9780064471060,0,English,38.3345,-86.4641,1,1996.0
